# Link Prediction 

Modeling network evolution:

* Preferential attachment model
* small world model

Link prediction: Given a network, can we predict which edges will be formed in the future?

![](images/4-4.1.png)

* What new edges are likly to form in this network?
* GIven a pair of nodes, how to assess whether they are likly to connect?

* **Triadic closure:** the tendency for people who sahre connections in a social network to become connected.


## Measure 1: Common Neighbors

THe number of common neighbors of nodes X and Y is 

$commonNeigh(X, Y) = |N(X)\bigcap{N(Y)}|$,

when $N(X)$ is the set of neighbors of node $X$

$commonNeigh(A, C) = |{B,D}| =2$

In [18]:
import networkx as nx
import matplotlib.pyplot as plt
import operator
%matplotlib widget

G=nx.Graph()
G.add_edges_from([('A','B'),('B','C'),('A','D'),('A','E'),('B','D'),('D','C'),('C','F'),('F','E'),('F','G'),('E','G'),('G','H'),('G','I')])
plt.figure(1)
pos = nx.spring_layout(G)
nx.draw_networkx(G, pos) 


FigureCanvasNbAgg()

In [19]:
common_neigh= [(e[0],e[1], len(list(nx.common_neighbors(G,e[0],e[1])))) for e in nx.non_edges(G)]
sorted(common_neigh , key= operator.itemgetter(2), reverse=True)
print(common_neigh)

[('A', 'C', 2), ('A', 'F', 1), ('A', 'H', 0), ('A', 'I', 0), ('A', 'G', 1), ('C', 'G', 1), ('C', 'H', 0), ('C', 'E', 1), ('C', 'I', 0), ('F', 'H', 1), ('F', 'D', 1), ('F', 'B', 1), ('F', 'I', 1), ('H', 'I', 1), ('H', 'D', 0), ('H', 'E', 1), ('H', 'B', 0), ('I', 'D', 0), ('I', 'E', 1), ('I', 'B', 0), ('E', 'D', 1), ('E', 'B', 1), ('D', 'G', 0), ('B', 'G', 0)]


Which edge would form more likely - A and G, H and I?

[('A', 'C', 2), ('A', 'F', 1), ('A', 'H', 0), ('A', 'I', 0), **('A', 'G', 1)**, ('C', 'G', 1), ('C', 'H', 0), ('C', 'E', 1), ('C', 'I', 0), ('F', 'H', 1), ('F', 'D', 1), ('F', 'B', 1), ('F', 'I', 1), **('H', 'I', 1)**, ('H', 'D', 0), ('H', 'E', 1), ('H', 'B', 0), ('I', 'D', 0), ('I', 'E', 1), ('I', 'B', 0), ('E', 'D', 1), ('E', 'B', 1), ('D', 'G', 0), ('B', 'G', 0)]

# Measure 2: Jaccard Coefficient

* Number of common neighbors normalized by the total number of neighbors

The Jaccard coefficient of nodes X and Y is

$jaccCoeff(X, Y) = \frac{N(X)\bigcap{N(Y)}}{N(X)\bigcup{N(Y)}}$

$jaccCoeff(A, C) = \frac{N(A)\bigcap{N(C)}}{N(A)\bigcup{N(C)}} = \frac{|\{B,D\}|}{|\{B, D, E, F\}|}= \frac{2}{4}= \frac{1}{2}$

![](images/4-4.2.png)

What is the Jaccard Coefficient between node A and F?

$jaccCoeff(A,F)=\frac{|\{E\}|}{|\{E,G,D,B,C\}|}=\frac{1}{5}$

In [21]:
L=list(nx.jaccard_coefficient(G))
L.sort(key=operator.itemgetter(2), reverse=True)
print(L)

[('H', 'I', 1.0), ('A', 'C', 0.5), ('F', 'H', 0.3333333333333333), ('F', 'I', 0.3333333333333333), ('H', 'E', 0.3333333333333333), ('I', 'E', 0.3333333333333333), ('A', 'F', 0.2), ('C', 'E', 0.2), ('F', 'D', 0.2), ('F', 'B', 0.2), ('E', 'D', 0.2), ('E', 'B', 0.2), ('A', 'G', 0.16666666666666666), ('C', 'G', 0.16666666666666666), ('A', 'H', 0.0), ('A', 'I', 0.0), ('C', 'H', 0.0), ('C', 'I', 0.0), ('H', 'D', 0.0), ('H', 'B', 0.0), ('I', 'D', 0.0), ('I', 'B', 0.0), ('D', 'G', 0.0), ('B', 'G', 0.0)]


H and I has higher Jaccard Coefficient, A and G has lower Jaccard Coefficient because I and H are both connected to a single neighbor which is a common neighbor G. Whereas the nodes A and G, they have one common neighbor, but they have more neighbors that are in the union of the neighbors of A and G. And so, therefore, they have a lower Jaccard coefficient. 

[**('H', 'I', 1.0)**, ('A', 'C', 0.5), ('F', 'H', 0.3333333333333333), ('F', 'I', 0.3333333333333333), ('H', 'E', 0.3333333333333333), ('I', 'E', 0.3333333333333333), ('A', 'F', 0.2), ('C', 'E', 0.2), ('F', 'D', 0.2), ('F', 'B', 0.2), ('E', 'D', 0.2), ('E', 'B', 0.2), **('A', 'G', 0.16666666666666666)**, ('C', 'G', 0.16666666666666666), ('A', 'H', 0.0), ('A', 'I', 0.0), ('C', 'H', 0.0), ('C', 'I', 0.0), ('H', 'D', 0.0), ('H', 'B', 0.0), ('I', 'D', 0.0), ('I', 'B', 0.0), ('D', 'G', 0.0), ('B', 'G', 0.0)]

## Measure 3: Resource Allocation

* Fraction of a "resource" taht a node can send to another through their common neighbors.

* The Resource Allocation index of nodes X and Y is

$resAlloc(X, Y) = \sum_{u\in N(X)\bigcap{N(Y)}}\frac{1}{|N(u)|}$

* if X and Y have a lot of common neighbors and they're going to have a large Resource Allocation index. 

* Z has n neighbors
* X sends 1 unit to Z, Z distributes the unit evenly among all neighbors
$\to$ Y receives 1/n of the unit.

![](images/4-4.3.png)

* **Example**: 

$\frac{1}{3}$ for degree of Node B, $\frac{1}{3}$  for degree of Node D, 
$redAlloc(A, C) = \frac{1}{3} +  \frac{1}{3} $

What is the Resource Allocation index of Node I and H?

Node I and H have only one common neighbor: G. The degree of node G is 4. Hence the Resource Allocation index is ¼ = 0.25.

In [23]:
L = list(nx.resource_allocation_index(G))
L.sort(key=operator.itemgetter(2),reverse=True)
print(L)

[('A', 'C', 0.6666666666666666), ('A', 'F', 0.3333333333333333), ('A', 'G', 0.3333333333333333), ('C', 'G', 0.3333333333333333), ('C', 'E', 0.3333333333333333), ('F', 'D', 0.3333333333333333), ('F', 'B', 0.3333333333333333), ('E', 'D', 0.3333333333333333), ('E', 'B', 0.3333333333333333), ('F', 'H', 0.25), ('F', 'I', 0.25), ('H', 'I', 0.25), ('H', 'E', 0.25), ('I', 'E', 0.25), ('A', 'H', 0), ('A', 'I', 0), ('C', 'H', 0), ('C', 'I', 0), ('H', 'D', 0), ('H', 'B', 0), ('I', 'D', 0), ('I', 'B', 0), ('D', 'G', 0), ('B', 'G', 0)]


[('A', 'C', 0.6666666666666666), ('A', 'F', 0.3333333333333333), **('A', 'G', 0.3333333333333333)**, ('C', 'G', 0.3333333333333333), ('C', 'E', 0.3333333333333333), ('F', 'D', 0.3333333333333333), ('F', 'B', 0.3333333333333333), ('E', 'D', 0.3333333333333333), ('E', 'B', 0.3333333333333333), ('F', 'H', 0.25), ('F', 'I', 0.25), **('H', 'I', 0.25)**, ('H', 'E', 0.25), ('I', 'E', 0.25), ('A', 'H', 0), ('A', 'I', 0), ('C', 'H', 0), ('C', 'I', 0), ('H', 'D', 0), ('H', 'B', 0), ('I', 'D', 0), ('I', 'B', 0), ('D', 'G', 0), ('B', 'G', 0)]

## Measure 4: Adamic Adar Index

* Similar to resouce allcoation index, but with log in the denominator.

* The Adamic-Adar index of nodes X and Y is 

$resAlloc(X, Y) = \sum_{u\in N(X)\bigcap{N(Y)}}\frac{1}{log(|N(u)|)}$

![](images/4-4.2.png)

$adamicAdar(A,C) =\frac{1}{log(3)}+\frac{1}{log(3)}$ 

In [24]:
L = list(nx.adamic_adar_index(G))
L.sort(key=operator.itemgetter(2),reverse=True)
print(L)

[('A', 'C', 1.8204784532536746), ('A', 'F', 0.9102392266268373), ('A', 'G', 0.9102392266268373), ('C', 'G', 0.9102392266268373), ('C', 'E', 0.9102392266268373), ('F', 'D', 0.9102392266268373), ('F', 'B', 0.9102392266268373), ('E', 'D', 0.9102392266268373), ('E', 'B', 0.9102392266268373), ('F', 'H', 0.7213475204444817), ('F', 'I', 0.7213475204444817), ('H', 'I', 0.7213475204444817), ('H', 'E', 0.7213475204444817), ('I', 'E', 0.7213475204444817), ('A', 'H', 0), ('A', 'I', 0), ('C', 'H', 0), ('C', 'I', 0), ('H', 'D', 0), ('H', 'B', 0), ('I', 'D', 0), ('I', 'B', 0), ('D', 'G', 0), ('B', 'G', 0)]



[('A', 'C', 1.8204784532536746), ('A', 'F', 0.9102392266268373), **('A', 'G', 0.9102392266268373)**, ('C', 'G', 0.9102392266268373), ('C', 'E', 0.9102392266268373), ('F', 'D', 0.9102392266268373), ('F', 'B', 0.9102392266268373), ('E', 'D', 0.9102392266268373), ('E', 'B', 0.9102392266268373), ('F', 'H', 0.7213475204444817), ('F', 'I', 0.7213475204444817), **('H', 'I', 0.7213475204444817)**, ('H', 'E', 0.7213475204444817), ('I', 'E', 0.7213475204444817), ('A', 'H', 0), ('A', 'I', 0), ('C', 'H', 0), ('C', 'I', 0), ('H', 'D', 0), ('H', 'B', 0), ('I', 'D', 0), ('I', 'B', 0), ('D', 'G', 0), ('B', 'G', 0)]

# Measure 5: Preferential Attachment

* In the preferential attachment model, nodes with high degree get more neighbors.

* Product of the nodes' degree
* The preferential attachment score of nodes X and Y is

$prefAttach(X,Y) = |N(X)||N(Y)|$

Pref_attach (A,C) = 3*3 =9

In [25]:
L = list(nx.preferential_attachment(G))
L.sort(key=operator.itemgetter(2),reverse=True)
print(L)

[('A', 'G', 12), ('C', 'G', 12), ('D', 'G', 12), ('B', 'G', 12), ('A', 'C', 9), ('A', 'F', 9), ('C', 'E', 9), ('F', 'D', 9), ('F', 'B', 9), ('E', 'D', 9), ('E', 'B', 9), ('A', 'H', 3), ('A', 'I', 3), ('C', 'H', 3), ('C', 'I', 3), ('F', 'H', 3), ('F', 'I', 3), ('H', 'D', 3), ('H', 'E', 3), ('H', 'B', 3), ('I', 'D', 3), ('I', 'E', 3), ('I', 'B', 3), ('H', 'I', 1)]


[**('A', 'G', 12)**, ('C', 'G', 12), ('D', 'G', 12), ('B', 'G', 12), ('A', 'C', 9), ('A', 'F', 9), ('C', 'E', 9), ('F', 'D', 9), ('F', 'B', 9), ('E', 'D', 9), ('E', 'B', 9), ('A', 'H', 3), ('A', 'I', 3), ('C', 'H', 3), ('C', 'I', 3), ('F', 'H', 3), ('F', 'I', 3), ('H', 'D', 3), ('H', 'E', 3), ('H', 'B', 3), ('I', 'D', 3), ('I', 'E', 3), ('I', 'B', 3), **('H', 'I', 1)**]

# Community Structure

* Some measures consider the community structure of the network for link prediction.
* Assume the nodes in this network belong to different communities (sets of ndoes).

![](images/4-4.4.png)

* Pairs of ndoes who belong to the same community and have many common neighbors in their community are likly to fom an edge.
* NUmber of common neighbors with bonus for neighbors in same community.
* The common Neighbor Soundarajan-Hopcroft score of nodes X and Y is

## Measure 6: Community Common Neighbors

$cnSoundarajanHopcroft(X, Y)= |N(X)\bigcap{N(Y)}| + \sum_{u \in N(X)\bigcap{N(u)}}F(u)$,

where $f(u) =1, u in same common as X and Y \\ 0, otherwise$


cnSoundarajanHopcroft(A, C)=2+2
cnSoundarajanHopcroft(E,I)=1+1
cnSoundarajanHopcroft(A, G)=1+0

In [27]:
# assign nodes to communities with attribute node 'community'
G.node['A']['community']=0
G.node['B']['community']=0
G.node['C']['community']=0
G.node['D']['community']=0
G.node['E']['community']=1
G.node['F']['community']=1
G.node['G']['community']=1
G.node['H']['community']=1
G.node['I']['community']=1


In [30]:
L= list(nx.cn_soundarajan_hopcroft(G))
L.sort(key=operator.itemgetter(2), reverse=True)
print(L)

[('A', 'C', 4), ('F', 'H', 2), ('F', 'I', 2), ('H', 'I', 2), ('H', 'E', 2), ('I', 'E', 2), ('A', 'F', 1), ('A', 'G', 1), ('C', 'G', 1), ('C', 'E', 1), ('F', 'D', 1), ('F', 'B', 1), ('E', 'D', 1), ('E', 'B', 1), ('A', 'H', 0), ('A', 'I', 0), ('C', 'H', 0), ('C', 'I', 0), ('H', 'D', 0), ('H', 'B', 0), ('I', 'D', 0), ('I', 'B', 0), ('D', 'G', 0), ('B', 'G', 0)]


[('A', 'C', 4), ('F', 'H', 2), ('F', 'I', 2), **('H', 'I', 2)**, ('H', 'E', 2), ('I', 'E', 2), ('A', 'F', 1), **('A', 'G', 1)**, ('C', 'G', 1), ('C', 'E', 1), ('F', 'D', 1), ('F', 'B', 1), ('E', 'D', 1), ('E', 'B', 1), ('A', 'H', 0), ('A', 'I', 0), ('C', 'H', 0), ('C', 'I', 0), ('H', 'D', 0), ('H', 'B', 0), ('I', 'D', 0), ('I', 'B', 0), ('D', 'G', 0), ('B', 'G', 0)]

# Measure 7: Community Resource Allocation

* Similar to resource allocation index, but only considering nodes in the same community

* The Resource Allcoation Soundarajan-Hopcroft score of nodes X and Y is:

$raSoundarajanHopcroft(X,Y)=\sum_{u\in{N(X)}\bigcap{N(Y)}}\frac{f(u)}{|N(u)|'}$

$f(u) = 1$ belongs to the community; 0 otherwise.

ra_soundarajan_hopcroft(A,C) = 1/3 + 1/3

ra_soundarajan_hopcroft(E, I) = 1/4

ra_soundarajan_hopcroft(A,G) =  0

In [34]:
L=list(nx.ra_index_soundarajan_hopcroft(G))
L.sort(key=operator.itemgetter(2), reverse=True)
print(L)

[('A', 'C', 0.6666666666666666), ('F', 'H', 0.25), ('F', 'I', 0.25), ('H', 'I', 0.25), ('H', 'E', 0.25), ('I', 'E', 0.25), ('A', 'F', 0), ('A', 'H', 0), ('A', 'I', 0), ('A', 'G', 0), ('C', 'G', 0), ('C', 'H', 0), ('C', 'E', 0), ('C', 'I', 0), ('F', 'D', 0), ('F', 'B', 0), ('H', 'D', 0), ('H', 'B', 0), ('I', 'D', 0), ('I', 'B', 0), ('E', 'D', 0), ('E', 'B', 0), ('D', 'G', 0), ('B', 'G', 0)]


* **All these measures are often used as features in the training data for ML**